In [1]:
import numpy as np
import matplotlib
from matplotlib import pyplot as plt

from IPython.display import display, clear_output
from ipywidgets import *

from magpie import magpie

In [2]:
# %matplotlib inline

# Lx = 1.1; Ly = 1.1; Lz = 5e-3
# ldim = [Lx, Ly, Lz]  # -- plate dimensions [x, y, z] in metres
# E = 9.0e+9           # -- Young's mod [Pa]
# rho = 8765           # -- density [kg/m^3]
# nu = 0.3             # -- poisson's ratio
# Nmodes = 4           # -- number of modes to compute
# h = np.sqrt(Lx * Ly) * 0.05   # -- grid spacing
# BCs = np.ones((4, 2)) * 1e15 # -- elastic constants around the edges
# m = 0; 

# chladni_plot = Output();


# def set_mode_number(b):
#     global m
#     if b.icon == 'arrow-left':
#         if m > 0:            
#             m -= 1
#             plot_mode_shapes()
#     elif b.icon == 'arrow-right':
#         if m < (Nmodes-1):
#             m += 1
#             plot_mode_shapes()
    

# def set_BCs(K0y, Kx0, KLy, KxL, R0y, Rx0, RLy, RxL):
#     global BCs
#     BCs = np.array([[Kx0,Rx0],[K0y,R0y],[KxL,RxL],[KLy,RLy]], dtype=np.float64)
#     plot_mode_shapes()

    
# def plot_mode_shapes():
#     """
#     """
    
#     with chladni_plot:
        
#         Q, Om, N, biharm = magpie(rho, E, nu, ldim, h, BCs, Nmodes)
#         for m in range(Nmodes):
#             mode_shapes = [np.real(np.reshape(Q[:, m], [N['x'], N['y']])) for m in range(Nmodes)]

#         clear_output(wait=True)
        
#         fig = plt.figure(figsize=(4,4))
#         ax = fig.add_subplot(111)

#         Z = abs(mode_shapes[m])
#         chladni = plt.pcolormesh(Z.T, cmap='copper_r', shading='gouraud')
#         ax.set_axis_off()
#         cmax = np.max(abs(mode_shapes[m])) * 0.25
#         chladni.set_clim(0.00, cmax)
        

#         plt.plot()        
#         plt.show()        

In [29]:
class MagpiePlot():
    
    def __init__(self):
        
        self.Q  = None
        self.Om = None
        self.N = None
        self.biharm = None
        self.mode_shapes = None
        

        self.Lx = 1.1; self.Ly = 1.1; self.Lz = 5e-3
        self.ldim = [self.Lx, self.Ly, self.Lz]  # -- plate dimensions [x, y, z] in metres
        self.E = 9.0e+9           # -- Young's mod [Pa]
        self.rho = 8765           # -- density [kg/m^3]
        self.nu = 0.3             # -- poisson's ratio
        self.Nmodes = 4           # -- number of modes to compute
        self.h = np.sqrt(self.Lx * self.Ly) * 0.05   # -- grid spacing
        self.BCs = np.ones((4, 2)) * 1e15 # -- elastic constants around the edges
        self.m = 0; 

        self.auto_plot = False

        self.chladni_plot = Output();
        self.generate_modes()
    
    def set_mode_number(self,b):
        m = self.m
        if b.icon == 'arrow-left':
            if m > 0:            
                m -= 1
        elif b.icon == 'arrow-right':
            if m < (self.Nmodes-1):
                m += 1
                
        if self.m != m:
            self.m = m
            if self.auto_plot:
                self.plot_mode_shapes()


    def set_BCs(self, K0y, Kx0, KLy, KxL, R0y, Rx0, RLy, RxL):        
        self.BCs = np.array([[Kx0,Rx0],[K0y,R0y],[KxL,RxL],[KLy,RLy]], dtype=np.float64)
        print(self.BCs)
        self.generate_modes()
        if self.auto_plot:
            self.plot_mode_shapes()
    
    def generate_modes(self):
        
        self.Q, self.Om, self.N, self.biharm = magpie(self.rho, self.E, self.nu, self.ldim, self.h, self.BCs, self.Nmodes)
        for m in range(self.Nmodes):
            self.mode_shapes = [np.real(np.reshape(self.Q[:, m], [self.N['x'], self.N['y']])) for m in range(self.Nmodes)]
    
    def plot_mode_shapes(self):
        """
        """
        m = self.m

        with self.chladni_plot:
            clear_output(wait=True)

            fig = plt.figure(figsize=(4,4))
            ax = fig.add_subplot(111)

            Z = abs(self.mode_shapes[m])
            chladni = plt.pcolormesh(Z.T, cmap='copper_r', shading='gouraud')
            ax.set_axis_off()
            cmax = np.max(Z) * 0.25
            chladni.set_clim(0.00, cmax)


            plt.plot()        
            plt.show()
            
magpie_plot = MagpiePlot() 

In [4]:
labels = ['K0y', 'Kx0', 'KLy', 'KxL', 'R0y', 'Rx0', 'RLy', 'RxL']

sliders =  {label:FloatLogSlider(
    value=0.0, base=10, min=0, max=15,
    description=label,
    continuous_update=True,
    orientation='vertical' if 'x' in label else 'horizontal',
    readout_format='.1e',
    layout=Layout(flex='1 1 1', width='auto')
) for label in labels}


# sliders['Rx0'] = FloatSlider(value=0.05, min=113.99043015, max=113.990430165, step=0.0000000001,
#                              readout=True, orientation='vertical')


Output()

In [5]:
form_item_layout = Layout(
    display='flex',
    flex_flow='row',
    justify_content='center'
)

form_items = [
    Box([Label(value='Dimensions'), 
         BoundedFloatText(min=0.2, max=2.0, description='X [m]'),
         BoundedFloatText(min=0.2, max=2.0, description='Y [m]'),
         BoundedFloatText(min=0.001, max=0.01, description='thickness [m]')], layout=form_item_layout),
    Box([Label(value='Material'), 
         Dropdown(options=["steel", "gold", "silver", "copper", "aluminium"])], layout=form_item_layout),
    Box([Label(value=''), 
         BoundedFloatText(min=0.2, max=2.0, description='Young\'s',disabled=True,),
         BoundedFloatText(min=0.2, max=2.0, description='Density',disabled=True,),
         BoundedFloatText(min=0.001, max=0.01, description='Poisson\'s',disabled=True,)], layout=form_item_layout),
    Box([Label(value='Resolution'), FloatSlider(value=0.05, min=0.005, max=0.05, step=0.001, 
                                                readout=True, readout_format='.3f',)], layout=form_item_layout)
]

form = Box(form_items, layout=Layout(
    display='flex',
    flex_flow='column',
    border='solid 2px',
    align_items='stretch',
    width='auto',
))



In [31]:
button_labels = ['arrow-left','arrow-right']
buttons ={label:widgets.Button(icon=label) for label in button_labels}
[b.on_click(magpie_plot.set_mode_number) for _,b in buttons.items()]


interactive_output(magpie_plot.set_BCs, sliders)
magpie_plot.auto_plot = True
magpie_plot.plot_mode_shapes()

display(
    VBox([form,
          VBox([sliders['KLy'],sliders['RLy']]),
          HBox([sliders['Kx0'],sliders['Rx0'],magpie_plot.chladni_plot,sliders['RxL'],sliders['KxL']]),
          VBox([sliders['R0y'],sliders['K0y']]),
          HBox([*buttons.values()])])
)